# Basic Setting

In [1]:
conda install tensorflow

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install keras

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import scipy

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Feature Extraction and split them into train and test set

In [5]:
filepath=os.path.dirname(os.path.dirname(os.path.realpath("main.ipynb")))  
# ...\\Spring2020-Project3-group2
os.chdir(filepath)

In [6]:
filepath

'C:\\Users'

If the above output is not correct. Please enter the filepath below to '..../Spring2020-Project3-group2/' manually. Thank you.

In [7]:
filepath = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/'
#####################   Modify above path   #########################

In [8]:
os.chdir(filepath)

# testpath=filepath+'/data/test_set/'
trainpath=filepath+'/data/train_set/'

# test_image_dir = testpath + "images/"
# test_pt_dir = testpath + "points/"
train_image_dir = trainpath + "images/"
train_pt_dir = trainpath + "points/"

In [9]:
import sklearn.metrics.pairwise
def pairwise_dist(vec):
    dist  = sklearn.metrics.pairwise_distances(vec, metric='euclidean')
    np.fill_diagonal(dist, np.nan)
    return dist
def feature(fiducial_pt_list,index):
    pairwise_dist_feature = pairwise_dist(fiducial_pt_list[index]).flatten()
    pairwise_dist_feature = pairwise_dist_feature[~np.isnan(pairwise_dist_feature)]
    return pairwise_dist_feature

In [10]:
f0 = time.time()
dataDir = train_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


X = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    X.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)
y =pd.read_csv(trainpath+'label.csv')['emotion_idx']
f1 = time.time()-f0

In [11]:
print("Feature Extraction time: %0.3fs" % (f1))

Feature Extraction time: 2.006s


In [12]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X,y,test_size=0.2,random_state=3662)

# Model

Here is the summary of all models we tried.

In [35]:
data = {'Model':['Bseline model:GBM','Knn', 'Improved GBM', 'XGboost','RandomForestClassifier','LogisticRegression', 'SVM','MLPClassifier','Final model: VotingClassifier'], 
        'Claimed Accuracy':['41.92%', '30.36%', '43.32%','47.12%',  '45.48%', '54.00%', '50.04%','49.44%','???'],
        'Training Time/s':['435.736s', '0.414s', '946.633s','119.812s',  '6.888s', '33.373s', '17.806s','298.493s','436.136s'],
       'Testing Time/s':['0.021s', '9.944s', '0.030s','0.189s',  '0.030s', '0.009s', '5.440s','0.173s','6.124s'],} 
pd.DataFrame(data)

,Model,Claimed Accuracy,Training Time/s,Testing Time/s
0,Bseline model:GBM,41.92%,435.736s,0.021s
1,Knn,30.36%,0.414s,9.944s
2,Improved GBM,43.32%,946.633s,0.030s
3,XGboost,47.12%,119.812s,0.189s
4,RandomForestClassifier,45.48%,6.888s,0.030s
5,LogisticRegression,54.00%,33.373s,0.009s
6,SVM,50.04%,17.806s,5.440s
7,MLPClassifier,49.44%,298.493s,0.173s
8,Final model: VotingClassifier,???,436.136s,6.124s


## Baseline model GBM (41.92%)

The baseline model we used is Boosted Decision Stumps.

In [14]:
gbm_baseline= GradientBoostingClassifier(n_estimators=100  , max_depth= 1,learning_rate=0.1)

In [15]:
model=gbm_baseline
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 435.736s; 
Training Accuracy : 78.1% 0.108s
Testing Accuracy : 40.4% 0.021s


In [16]:
print(classification_report(test_y_dis,pred))

              precision    recall  f1-score   support

           1       0.50      0.67      0.57        24
           2       0.65      0.71      0.68        24
           3       0.34      0.36      0.35        28
           4       0.23      0.41      0.30        17
           5       0.58      0.44      0.50        25
           6       0.25      0.20      0.22        20
           7       0.50      0.31      0.38        26
           8       0.66      0.86      0.75        22
           9       0.56      0.47      0.51        19
          10       0.50      0.26      0.34        31
          11       0.40      0.45      0.43        22
          12       0.25      0.29      0.27        21
          13       0.31      0.15      0.21        26
          14       0.47      0.74      0.57        19
          15       0.29      0.45      0.36        11
          16       0.60      0.62      0.61        24
          17       0.36      0.40      0.38        30
          18       0.39    

## Advanced model

Then, let us find some other advanced model.

### Knn (30.36%)

We use Knn first. But the accuracy is not acceptable.

In [17]:
knn=KNeighborsClassifier(n_neighbors=24)

In [18]:
model=knn
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 0.414s; 
Training Accuracy : 42.4% 39.880s
Testing Accuracy : 30.4% 9.944s


### Improved GBM (43.32%)

We try to improve our baseline model by increasing the "max_depth". Its accuracy is improved but the fitting time is longer.

In [19]:
gbm_improved= GradientBoostingClassifier(n_estimators=100  , max_depth= 2,learning_rate=0.1)

In [20]:
model=gbm_improved
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 946.633s; 
Training Accuracy : 99.1% 0.164s
Testing Accuracy : 43.6% 0.030s


Moreover, we try some advanced model, including XGBoost, RandomForestClassifier, LogisticRegression, SVM, MLPclassifier. Their accuracy and fitting time are both improved compared with baseline model. 

### XGboost(47.12%)

In [21]:
xgboost_model_final =xgboost.XGBClassifier(max_depth=4,n_estimators=50,learning_rate=0.1,
                       min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,reg_alpha=0.005)

In [22]:
model=xgboost_model_final
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 119.812s; 
Training Accuracy : 99.8% 0.728s
Testing Accuracy : 46.8% 0.189s


### RandomForestClassifier (45.48%)

In [23]:
randomforest_model_final=RandomForestClassifier(n_estimators = 100, criterion = 'gini', 
                             random_state = 42, min_samples_leaf=1, max_features='sqrt')

In [24]:
model=randomforest_model_final
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 6.888s; 
Training Accuracy : 100.0% 0.116s
Testing Accuracy : 42.0% 0.030s


### LogisticRegression (54.00%)

In [25]:
logistic_model_final =LogisticRegression(C=0.01,
                                         dual=False,
                                         fit_intercept=True,
                                         intercept_scaling=1,
                                         max_iter=300,
                                         multi_class='multinomial',
                                         penalty='l2',
                                         solver='newton-cg',
                                         tol=0.0001)

In [26]:
model=logistic_model_final
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 33.373s; 
Training Accuracy : 82.7% 0.030s
Testing Accuracy : 55.4% 0.009s


### SVM (50.04%)

In [27]:
svm_model_final =SVC(C=0.1,decision_function_shape='ovr',degree=2,gamma=0.1,kernel='linear')

In [28]:
model=svm_model_final
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 17.806s; 
Training Accuracy : 100.0% 21.808s
Testing Accuracy : 48.6% 5.440s


### Neural Network-MLPClassifier (49.44%)

In [29]:
MLP_model_final =MLPClassifier(early_stopping=True,
                              hidden_layer_sizes=(3000,),
                              learning_rate='adaptive',
                              solver='lbfgs',
                              validation_fraction=0.2)

In [30]:
model=MLP_model_final
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 298.493s; 
Training Accuracy : 100.0% 0.576s
Testing Accuracy : 48.6% 0.173s


## Final model: VotingClassifier

Finally, we use VotingClassifier to combine the top models together as the final model.

In [31]:
voting_clf = VotingClassifier(
    estimators=[('rf_clf',randomforest_model_final),
                ('log_clf', logistic_model_final),
                ('svm_clf', svm_model_final),
               ('xgb_clf', xgboost_model_final),
                ('MLP_clf',MLP_model_final)],
    voting='hard',
#    weights=[0.45,0.54,0.50,0.47,0.49],
    
)

In [33]:
model=voting_clf
t0 = time.time()
model.fit(train_x_dis,train_y_dis)
t1 = time.time() 
training_acc=model.score(train_x_dis,train_y_dis)
t2 = time.time() 
pred=model.predict(test_x_dis)
t3 = time.time() 
testing_acc=model.score(test_x_dis,test_y_dis)

print("Model fit time: %0.3fs; " % (t1-t0))
print("Training Accuracy : %0.1f%% %0.3fs" % (training_acc*100,t2-t1))
print("Testing Accuracy : %0.1f%% %0.3fs" % (testing_acc*100,t3-t2))

Model fit time: 436.136s; 
Training Accuracy : 100.0% 23.888s
Testing Accuracy : 53.0% 6.124s


In [34]:
print(classification_report(test_y_dis,model.predict(test_x_dis)))

              precision    recall  f1-score   support

           1       0.57      0.83      0.68        24
           2       0.75      0.75      0.75        24
           3       0.44      0.50      0.47        28
           4       0.34      0.65      0.45        17
           5       0.77      0.80      0.78        25
           6       0.53      0.50      0.51        20
           7       0.50      0.54      0.52        26
           8       0.70      0.95      0.81        22
           9       0.65      0.58      0.61        19
          10       0.65      0.42      0.51        31
          11       0.57      0.55      0.56        22
          12       0.38      0.38      0.38        21
          13       0.33      0.19      0.24        26
          14       0.52      0.74      0.61        19
          15       0.38      0.55      0.44        11
          16       0.73      0.79      0.76        24
          17       0.54      0.43      0.48        30
          18       0.43    